In [2]:
import cv2
import mediapipe as mp
import time
import numpy as np
import math
from pynput.mouse import Controller
from pynput.mouse import Button



In [3]:
class handDetector():
    def __init__(self, mode=False, maxHands=2, detectionCon=1, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,
                                        self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,
                                               self.mpHands.HAND_CONNECTIONS)

        return img

    def findPosition(self, img, handNo=0, draw=True):
        xList = []
        yList = []
        bbox = []
        self.lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                xList.append(cx)
                yList.append(cy)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
Lllllllllllllllllloolllllllllllllllllllllllllllllllllllllll.....................????
            if draw:
                cv2.rectangle(img, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20),
                              (0, 255, 0), 2)

        return self.lmList, bbox

    def fingersUp(self):
        fingers = []
        if len(self.lmList) >= 4:
            # Thumb
            if self.lmList[self.tipIds[0]][1] > self.lmList[self.tipIds[0] - 1][1]:
                fingers.append(1)
            else:
                fingers.append(0)

            # Fingers
            for id in range(1, 5):
                if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id] - 2][2]:
                    fingers.append(1)
                else:
                    fingers.append(0)

        return fingers

    def findDistance(self, p1, p2, img, draw=True, r=15, t=3):
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), t)
            cv2.circle(img, (x1, y1), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (cx, cy), r, (0, 0, 255), cv2.FILLED)
        length = math.hypot(x2 - x1, y2 - y1)

        return length, img, [x1, y1, x2, y2, cx, cy]

In [7]:
import cv2
import numpy as np
import time
import pyautogui
pyautogui.FAILSAFE = False

##########################
wCam, hCam = 640, 480
frameR = 100 # Frame Reduction
smoothening = 5
#########################

pTime = 0
plocX, plocY = 0, 0
clocX, clocY = 0, 0

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, wCam)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, hCam)
wScr, hScr = pyautogui.size()
detector = handDetector()
mouse = Controller()

   


while True:
    # 1. Find hand Landmarks
    success, img = cap.read()
    img = detector.findHands(img)
    lmList, bbox = detector.findPosition(img, draw=True)
    # 2. Get the tip of the index and middle fingers
    if len(lmList) != 0:
        x1, y1 = lmList[8][1:]
        x2, y2 = lmList[12][1:]
        # print(x1, y1, x2, y2)

    # 3. Check which fingers are up
    fingers = detector.fingersUp()
    # print(fingers)
    cv2.rectangle(
        img, (frameR, frameR), (wCam - frameR, hCam - frameR), (255, 0, 255), 2
    )
    # 4. Only Index Finger : Moving Mode
    if len(fingers) >= 2 and fingers[1] == 1 and fingers[2] == 0:

        # 5. Convert Coordinates
        x3 = np.interp(x1, (frameR, wCam - frameR), (0, wScr))
        y3 = np.interp(y1, (frameR, hCam - frameR), (0, hScr - frameR))

        # 6. Smoothen Values
        clocX = plocX + (x3 - plocX) / smoothening
        clocY = plocY + (y3 - plocY) / smoothening

        # 7. Move Mouse
        mouse.position = (1 * (wScr - clocX), 1 * clocY)
        cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
        plocX, plocY = clocX, clocY

    # 8. Both Index and middle fingers are up : Clicking Mode
    if len(fingers) >= 2 and fingers[1] == 1 and fingers[2] == 1:

        # 9. Find distance between fingers
        length, img, lineInfo = detector.findDistance(8, 12, img)
        print(length)
        # 10. Click mouse if distance is short
        if length < 40:
            cv2.circle(img, (lineInfo[4], lineInfo[5]), 15, (0, 255, 0), cv2.FILLED)
            mouse.click(button=Button.left, count=1)

    # 11. Frame Rate
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(
        img,
        str(int(fps)),
        (20, 50),
        cv2.FONT_HERSHEY_PLAIN,
        3,
        (255, 0, 0),
        3,
    )
    # 12. Display
    cv2.imshow("Image", img)
    key_=cv2.waitKey(1)
    if key_ == ord('q'):
        break
cv2.destroyAllWindows()
cap.release()

2.23606797749979
12.0
8.246211251235321
11.40175425099138
6.708203932499369
5.656854249492381
3.6055512754639896
4.47213595499958
1.0
4.123105625617661
8.06225774829855
9.848857801796104
12.0
49.49747468305833
49.64876634922564
44.011362169330766
42.720018726587654
42.941821107167776
42.04759208325728
43.86342439892262
45.31004303683677
15.264337522473749
20.12461179749811
19.4164878389476
18.439088914585778
19.235384061671347
15.297058540778355
15.231546211727817
15.231546211727817
21.470910553583888
18.788294228055936
52.55473337388365
53.851648071345046
53.851648071345046
53.851648071345046
52.55473337388365
48.466483264210545
41.78516483155236
48.38388161361178
51.54609587543949
49.64876634922564
48.703182647543684
52.201532544552755
53.141321022345686
51.54609587543949
50.159744815937806
50.566787519082126
50.566787519082126
50.99019513592785
50.566787519082126
49.65883607174055
52.0
52.55473337388365
52.92447448959697
51.478150704935
53.31041174104736
51.07837115648854
51.0783711

61.98386886924693
62.297672508690084
60.74537019394976
60.74537019394976
59.77457653551382
58.821764679410975
58.52349955359813
60.40695324215582
53.75872022286245
57.27128425310541
54.56189146281496
54.120236510939236
55.47071299343465
55.56977595779922
54.203320931470614
55.2268050859363
54.08326913195984
43.266615305567875
39.408120990476064
36.05551275463989
34.20526275297414
33.60059523282288
31.400636936215164
46.32493928760187
44.1021541423999
45.48626166217663
47.70744176750625
46.32493928760187
43.266615305567875
39.408120990476064
35.805027579936315
30.479501308256342
26.870057685088806
23.430749027719965
22.80350850198276
23.60084744241189
22.203603311174515
22.80350850198276
22.203603311174515
22.203603311174515
24.758836806279895
33.83784863137726
33.83784863137726
48.02082881417187
48.373546489791295
47.01063709417263
48.76474136094643
47.885279575251516
45.17742799230607
41.14608122288197
30.675723300355937
41.10960958218894
21.023796041628636
28.460498941515414
39.39543

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [2]:
pip install pynput


     -------------------------------------- 89.2/89.2 kB 422.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install pyautogui

     -------------------------------------- 61.2/61.2 kB 822.5 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     -------------------------------------- 168.2/168.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: fini